In [2]:
import pandas as pd

In [3]:
#Global paramenters
#API URLs from RxNorm
base_endpoint = "https://api.fda.gov/drug/ndc.json?search=active_ingredients.name:"
limit = "&limit=1000"
#dataframe
df = pd.DataFrame()

In [4]:
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    df = pd.read_json(base_endpoint+name_of_drug+limit, orient='values', typ='series', convert_dates=False)
    new_df = pd.DataFrame(df.results)
    return new_df

In [11]:
ingredient_df = pd.read_csv("ingredients.csv")
ingredients =  ingredient_df["Ingredient"].tolist()

In [12]:
df_opioid = pd.DataFrame()
for drug in ingredients:
    merg_df = query_drug(drug)
    merg_df["active_ingredient"] = drug
    df_opioid = pd.concat([df_opioid, merg_df]).reset_index(drop=True)
df_opioid

,product_ndc,generic_name,labeler_name,dea_schedule,active_ingredients,finished,packaging,listing_expiration_date,openfda,marketing_category,...,marketing_start_date,product_id,brand_name_base,brand_name,route,application_number,pharm_class,active_ingredient,marketing_end_date,brand_name_suffix
0,49812-0033,Alfentanil HCl,"Veranova, L.P.",CII,"[{'name': 'ALFENTANIL HYDROCHLORIDE', 'strengt...",False,"[{'package_ndc': '49812-0033-1', 'description'...",20231231,{},BULK INGREDIENT,...,19970919,49812-0033_ef2a5a7a-737a-2610-e053-2995a90af9e7,None,NaN,NaN,NaN,NaN,alfentanil,NaN,NaN
1,17478-067,Alfentanil Hydrochloride,Akorn,CII,"[{'name': 'ALFENTANIL HYDROCHLORIDE', 'strengt...",True,"[{'package_ndc': '17478-067-02', 'description'...",20231231,"{'manufacturer_name': ['Akorn'], 'rxcui': ['17...",NDA,...,20100201,17478-067_aea7d858-c41e-4726-8678-9c2bf08e5d56,Alfentanil Hydrochloride,Alfentanil Hydrochloride,[INTRAVENOUS],NDA019353,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",alfentanil,NaN,NaN
2,10702-343,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",CII,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '10702-343-01', 'description'...",20231231,"{'manufacturer_name': ['KVK-Tech, Inc.'], 'rxc...",NDA AUTHORIZED GENERIC,...,20190628,10702-343_c2b2ecba-e068-2abc-e053-2a95a90ae68c,Benzhydrocodone and Acetaminophen,Benzhydrocodone and Acetaminophen,[ORAL],NDA208653,NaN,benzhydrocodone,NaN,NaN
3,10702-340,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",CII,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '10702-340-01', 'description'...",20231231,"{'manufacturer_name': ['KVK-Tech, Inc.'], 'rxc...",NDA,...,20190628,10702-340_bd5dd034-4b41-2f37-e053-2995a90a4f77,APADAZ,APADAZ,[ORAL],NDA208653,NaN,benzhydrocodone,NaN,NaN
4,10702-341,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",CII,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '10702-341-01', 'description'...",20231231,"{'manufacturer_name': ['KVK-Tech, Inc.'], 'rxc...",NDA,...,20190628,10702-341_bd5dd034-4b41-2f37-e053-2995a90a4f77,APADAZ,APADAZ,[ORAL],NDA208653,NaN,benzhydrocodone,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,71335-0111,Tramadol Hydrochloride and Acetaminophen,Bryant Ranch Prepack,CIV,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '71335-0111-1', 'description'...",20231231,{'manufacturer_name': ['Bryant Ranch Prepack']...,ANDA,...,20091217,71335-0111_0950e072-2f5b-4f10-b7b3-5607ce7b6822,Tramadol Hydrochloride and Acetaminophen,Tramadol Hydrochloride and Acetaminophen,[ORAL],ANDA090485,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",tramadol,NaN,NaN
2012,70518-1254,tramadol hydrochloride,REMEDYREPACK INC.,CIV,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '70518-1254-1', 'description'...",20241231,"{'manufacturer_name': ['REMEDYREPACK INC.'], '...",ANDA,...,20180613,70518-1254_fd7b9bab-b42f-a258-e053-6294a90a3a56,tramadol hydrochloride,tramadol hydrochloride,[ORAL],ANDA075964,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",tramadol,NaN,NaN
2013,71205-569,TRAMADOL HYDROCHLORIDE,Proficient Rx LP,CIV,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '71205-569-30', 'description'...",20231231,"{'manufacturer_name': ['Proficient Rx LP'], 'r...",ANDA,...,20140819,71205-569_f234949f-7ee3-4778-9875-4c3ce51e8b38,TRAMADOL HYDROCHLORIDE,TRAMADOL HYDROCHLORIDE,[ORAL],ANDA200503,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",tramadol,NaN,NaN
2014,60219-2348,Tramadol Hydrochloride,Amneal Pharmaceuticals NY LLC,CIV,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '60219-2348-1', 'description'...",20241231,{'manufacturer_name': ['Amneal Pharmaceuticals...,ANDA,...,20230213,60219-2348_1d10f961-04a4-4332-a2ba-86ba99ba027e,Tramadol Hydrochloride,Tramadol Hydrochloride,[ORAL],ANDA076003,"[Full Opioid Agonists [MoA], Opioid

In [13]:
#get within package
df_package = pd.DataFrame()
i = 0
while i < df_opioid.shape[0]:
    df_temp = pd.DataFrame(df_opioid.packaging[i])
    df_temp.insert(0,"generic_name", df_opioid.generic_name[i])
    df_temp.insert(1,"labeler_name", df_opioid.labeler_name[i])
    df_temp.insert(2,"brand_name", df_opioid.brand_name[i])
    df_temp = df_temp.assign(active_ingredient=df_opioid.active_ingredient[i])
    df_package = pd.concat([df_package, df_temp]).drop_duplicates(subset=['package_ndc']).reset_index(drop=True)
    i+=1
df_package

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,active_ingredient,sample,marketing_end_date
0,Alfentanil HCl,"Veranova, L.P.",NaN,49812-0033-1,1 CONTAINER in 1 CONTAINER (49812-0033-1) / 1...,19-SEP-97,alfentanil,NaN,NaN
1,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478-067-02,10 AMPULE in 1 CARTON (17478-067-02) / 2 mL i...,20100201,alfentanil,False,NaN
2,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478-067-05,10 AMPULE in 1 CARTON (17478-067-05) / 5 mL i...,20100201,alfentanil,False,NaN
3,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",Benzhydrocodone and Acetaminophen,10702-343-01,"100 TABLET in 1 BOTTLE, PLASTIC (10702-343-01)",20190628,benzhydrocodone,False,NaN
4,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",APADAZ,10702-340-01,"100 TABLET in 1 BOTTLE, PLASTIC (10702-340-01)",20190628,benzhydrocodone,False,NaN
...,...,...,...,...,...,...,...,...,...
4652,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090-0806-3,100 TABLET in 1 BOTTLE (50090-0806-3),20141128,tramadol,False,NaN
4653,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090-0806-4,15 TABLET in 1 BOTTLE (50090-0806-4),20141128,tramadol,False,NaN
4654,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090-0806-5,60 TABLET in 1 BOTTLE (50090-0806-5),20141128,tramadol,False,NaN
4655,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090-0806-7,45 TABLET in 1 BOTTLE (50090-0806-7),20140815,tramadol,False,NaN


In [14]:
#Change to NDC11
for ele in df_package.package_ndc:
    s = ele.split("-")
    S = []
    for i, L in enumerate([5,4,2]):
        if len(s[i]) < L:
            S.append('0'*(L-len(s[i])) + s[i])
        else:
            S.append(s[i])
    ndc11 = ''.join(S)
    df_package.replace(ele,ndc11, inplace=True)
df_package

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,active_ingredient,sample,marketing_end_date
0,Alfentanil HCl,"Veranova, L.P.",NaN,49812003301,1 CONTAINER in 1 CONTAINER (49812-0033-1) / 1...,19-SEP-97,alfentanil,NaN,NaN
1,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478006702,10 AMPULE in 1 CARTON (17478-067-02) / 2 mL i...,20100201,alfentanil,False,NaN
2,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478006705,10 AMPULE in 1 CARTON (17478-067-05) / 5 mL i...,20100201,alfentanil,False,NaN
3,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",Benzhydrocodone and Acetaminophen,10702034301,"100 TABLET in 1 BOTTLE, PLASTIC (10702-343-01)",20190628,benzhydrocodone,False,NaN
4,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",APADAZ,10702034001,"100 TABLET in 1 BOTTLE, PLASTIC (10702-340-01)",20190628,benzhydrocodone,False,NaN
...,...,...,...,...,...,...,...,...,...
4652,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090080603,100 TABLET in 1 BOTTLE (50090-0806-3),20141128,tramadol,False,NaN
4653,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090080604,15 TABLET in 1 BOTTLE (50090-0806-4),20141128,tramadol,False,NaN
4654,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090080605,60 TABLET in 1 BOTTLE (50090-0806-5),20141128,tramadol,False,NaN
4655,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090080607,45 TABLET in 1 BOTTLE (50090-0806-7),20140815,tramadol,False,NaN


In [15]:
df_package.to_excel("ndc_opioids.xlsx", index=False)

In [16]:
df_cdc = pd.read_excel("CDC_opioid NDC_oral MME conversion_update_2020.xlsx", sheet_name="Opioids", dtype='object')
df_cdc

,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
15294,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
15295,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
15296,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
15297,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [17]:
#Identify new drugs found
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame()
new_drug = pd.DataFrame()
for code in df_package['package_ndc']:
    if df_cdc.isin([str(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = pd.concat([same_drug, df_package.loc[df_package['package_ndc']==code]]).reset_index(drop=True)
    else:
        new_drug_count += 1
        new_drug = pd.concat([new_drug, df_package.loc[df_package['package_ndc']==code]]).reset_index(drop=True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  1653
New drug count:  3004


In [18]:
absent_drugs = pd.DataFrame()
count = 0
#for every code from the combind excel list
for code in df_cdc['NDC']:
    if df_package.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = pd.concat([absent_drugs, df_cdc.loc[df_cdc['NDC']==code]]).reset_index(drop=True)
print('Count: ', count)
absent_drugs

Count:  1653


,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
13641,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
13642,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
13643,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
13644,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [19]:
#export
same_drug.to_excel("Same_Opioid_Medications.xlsx", index=False)
new_drug.to_excel("New_Opioids_Added.xlsx", index=False)
absent_drugs.to_excel("Absent_Opioid.xlsx", index=False)

In [20]:
absent_drugs

,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
13641,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
13642,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
13643,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
13644,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [5]:
#Start of nonpain
df_nonpain = pd.read_excel("ndc_opioids.xlsx")
df_nonpain

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,active_ingredient,sample,marketing_end_date
0,Alfentanil HCl,"Veranova, L.P.",NaN,49812003301,1 CONTAINER in 1 CONTAINER (49812-0033-1) / 1...,19-SEP-97,alfentanil,NaN,NaN
1,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478006702,10 AMPULE in 1 CARTON (17478-067-02) / 2 mL i...,20100201,alfentanil,0.0,NaN
2,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478006705,10 AMPULE in 1 CARTON (17478-067-05) / 5 mL i...,20100201,alfentanil,0.0,NaN
3,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",Benzhydrocodone and Acetaminophen,10702034301,"100 TABLET in 1 BOTTLE, PLASTIC (10702-343-01)",20190628,benzhydrocodone,0.0,NaN
4,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",APADAZ,10702034001,"100 TABLET in 1 BOTTLE, PLASTIC (10702-340-01)",20190628,benzhydrocodone,0.0,NaN
...,...,...,...,...,...,...,...,...,...
4652,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090080603,100 TABLET in 1 BOTTLE (50090-0806-3),20141128,tramadol,0.0,NaN
4653,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090080604,15 TABLET in 1 BOTTLE (50090-0806-4),20141128,tramadol,0.0,NaN
4654,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090080605,60 TABLET in 1 BOTTLE (50090-0806-5),20141128,tramadol,0.0,NaN
4655,tramadol hydrochloride,A-S Medication Solutions,tramadol hydrochloride,50090080607,45 TABLET in 1 BOTTLE (50090-0806-7),20140815,tramadol,0.0,NaN


In [17]:
drug_ingredients = pd.read_csv("ingredients.csv")
drug_ingredients

,Ingredient,NON-Pain
0,alfentanil,NaN
1,benzhydrocodone,NaN
2,buprenorphine,NaN
3,butorphanol,NaN
4,codeine,NaN
5,difenoxin,True
6,dihydrocodeine,NaN
7,diphenoxylate,NaN
8,fentanyl,NaN
9,hydrocodone,NaN


In [34]:
# Reading the Excel file
df_nonpain = pd.read_excel("ndc_opioids.xlsx")
drug_ingredients = pd.read_csv("ingredients.csv")

nonpainIngredients = drug_ingredients[(drug_ingredients['NON-Pain'] == True)]
nonpainIngredients = nonpainIngredients['Ingredient'].to_numpy()

# Filtering the main DataFrame for non-pain ingredients
condition1 = df_nonpain['active_ingredient'].isin(nonpainIngredients)

# Separate condition for 'opium' ingredient
opium_condition = (df_nonpain['active_ingredient'].str.lower() == "opium")
opium_tincture_condition = opium_condition & \
                           df_nonpain['generic_name'].str.contains("tincture", case=False, na=False)

df_nonpain = df_nonpain[(condition1 & ~opium_condition) | opium_tincture_condition]
df_nonpain

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,active_ingredient,sample,marketing_end_date
767,difenoxin and atropine sulfate,Sebela Pharmaceuticals Inc.,Motofen,54766020004,4 TABLET in 1 BLISTER PACK (54766-200-04),20170411,difenoxin,1.0,NaN
768,difenoxin and atropine sulfate,Sebela Pharmaceuticals Inc.,Motofen,54766020010,100 TABLET in 1 BOTTLE (54766-200-10),20170411,difenoxin,0.0,NaN
2792,Noscapine,SpecGx LLC,NaN,406488706,50 kg in 1 DRUM (0406-4887-06),31-DEC-12,noscapine,NaN,NaN
2793,Noscapine,SpecGx LLC,NaN,406488722,5 kg in 1 DRUM (0406-4887-22),31-DEC-12,noscapine,NaN,NaN
2794,Noscapine,SpecGx LLC,NaN,406488765,50 kg in 1 DRUM (0406-4887-65),31-DEC-12,noscapine,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3984,"Octocrylene, Avobenzone, Ethylhexyl Salicylate...",Pella Pharmaceuticals Co. Ltd,Derma Pella Sunscreen Advance Cream-Gel,82160072601,1 TUBE in 1 CARTON (82160-726-01) / 50 g in 1...,20221226,tetrapeptide-15,0.0,NaN
3985,"Eucalyptus Globulus, Ipecacuanha, Aconitum Nap...","Deseret Biologicals, Inc.",Influenza Plus,43742155901,"30 mL in 1 BOTTLE, DROPPER (43742-1559-1)",20190910,tetrapeptide-15,0.0,20240912
3986,"Allium Sativum, Echinacea (Angustifolia), Gink...","Deseret Biologicals, Inc.",Viracomp,43742156701,"30 mL in 1 BOTTLE, DROPPER (43742-1567-1)",20200103,tetrapeptide-15,0.0,20250513
3987,"Influenzinum (2019-2020), Herpes Simplex 1 Nos...","Deseret Biologicals, Inc.",Virus Combination,43742158601,"30 mL in 1 BOTTLE, DROPPER (43742-1586-1)",20191025,tetrapeptide-15,0.0,20250908


In [36]:
df_nonpain.reset_index(drop=True, inplace=True)
df_nonpain

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,active_ingredient,sample,marketing_end_date
0,difenoxin and atropine sulfate,Sebela Pharmaceuticals Inc.,Motofen,54766020004,4 TABLET in 1 BLISTER PACK (54766-200-04),20170411,difenoxin,1.0,NaN
1,difenoxin and atropine sulfate,Sebela Pharmaceuticals Inc.,Motofen,54766020010,100 TABLET in 1 BOTTLE (54766-200-10),20170411,difenoxin,0.0,NaN
2,Noscapine,SpecGx LLC,NaN,406488706,50 kg in 1 DRUM (0406-4887-06),31-DEC-12,noscapine,NaN,NaN
3,Noscapine,SpecGx LLC,NaN,406488722,5 kg in 1 DRUM (0406-4887-22),31-DEC-12,noscapine,NaN,NaN
4,Noscapine,SpecGx LLC,NaN,406488765,50 kg in 1 DRUM (0406-4887-65),31-DEC-12,noscapine,NaN,NaN
...,...,...,...,...,...,...,...,...,...
62,"Octocrylene, Avobenzone, Ethylhexyl Salicylate...",Pella Pharmaceuticals Co. Ltd,Derma Pella Sunscreen Advance Cream-Gel,82160072601,1 TUBE in 1 CARTON (82160-726-01) / 50 g in 1...,20221226,tetrapeptide-15,0.0,NaN
63,"Eucalyptus Globulus, Ipecacuanha, Aconitum Nap...","Deseret Biologicals, Inc.",Influenza Plus,43742155901,"30 mL in 1 BOTTLE, DROPPER (43742-1559-1)",20190910,tetrapeptide-15,0.0,20240912
64,"Allium Sativum, Echinacea (Angustifolia), Gink...","Deseret Biologicals, Inc.",Viracomp,43742156701,"30 mL in 1 BOTTLE, DROPPER (43742-1567-1)",20200103,tetrapeptide-15,0.0,20250513
65,"Influenzinum (2019-2020), Herpes Simplex 1 Nos...","Deseret Biologicals, Inc.",Virus Combination,43742158601,"30 mL in 1 BOTTLE, DROPPER (43742-1586-1)",20191025,tetrapeptide-15,0.0,20250908


In [37]:
df_nonpain.to_excel("non-painOpioids.xlsx", index=False)